Name: Arjun Bhan                   
UNI: AB5666

# MNIST MLP Digit Recognition Network

For this problem, you will code a basic digit recognition network. The data are images which specify the digits 1 to 10 as (1, 28, 28) data - this data is black and white images. Each pixed of the image is an intensity between 0 and 255, and together the (1, 28, 28) pixel image can be visualized as a picture of a digit. The data is given to you as $\{(x^{(i)}, y^{(i)})\}_{i=1}^{N}$ where $y$ is the given label and x is the (1, 28, 28) data. This data will be gotten from `torchvision`, a repository of computer vision data and models.

Highlevel, the model and notebook goes as follows:
*   You first download the data and specify the batch size of B = 16. Each image will need to be turned from a (1, 28, 28) volume into a vector of dimension 784 = 1 * 28 * 28. So each batch will be of size (16, 784).
*   Then, you pass the model through two hidden layers, one of dimension (784, 32) and another of dimension (32, 16). After each linear map, you pass the data through a TanH nonlinearity.
*   Finally, you pass the data through a (32, 10) linear layer and you return the log softmax of the data.
*   What objective do you use? Be careful!
*   How do you compute accuracy both manually and with torchmetrics?
*   How do you compute AUROC?

Accuracy should be higher than 85%. If you use another nonlinearity, like ReLU, you might get higher. Play around with this but submit working code that does better than 85%.







In [ ]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 4.7 MB/s eta 0:00:00


In [ ]:
import torchvision
from torchmetrics import Accuracy
from torchvision import transforms
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torchmetrics

In [ ]:

SEED = 1
torch.manual_seed(SEED)

In [ ]:
image_path = './'


transform = transforms.Compose([
      transforms.ToTensor()
])

mnist_train_dataset = torchvision.datasets.MNIST(
    root=image_path,
    train=True,
    transform = transform,
    download=True
  )

mnist_test_dataset = torchvision.datasets.MNIST(
    root=image_path,
    train=False,
    transform=transform,
    download=False
)

100%|██████████| 9912422/9912422 [00:00<00:00, 20555791.02it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 38237277.09it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 29950463.15it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 17558091.03it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [ ]:
BATCH_SIZE = 64
LR = 0.001
EPOCHS = 20
train_dl =  DataLoader(mnist_train_dataset, batch_size = BATCH_SIZE, shuffle = True)
test_dl =  DataLoader(mnist_test_dataset, batch_size = BATCH_SIZE, shuffle = True)

In [ ]:
class MLPClassifier(nn.Module):

  def __init__(self):
    super().__init__()
    self.linear1 =  nn.Linear(784, 32)
    self.linear2 = nn.Linear(32, 16)
    self.linear3 = nn.Linear(16, 10)

  def forward(self, x):
    x = x.reshape(-1,784)

    x = self.linear1(x)

    x = nn.Tanh()(x)

    x = self.linear2(x)

    x = nn.Tanh()(x)

    x = self.linear3(x)


    return nn.functional.log_softmax(x, dim=1)

model = MLPClassifier()

In [ ]:
loss_fn = torch.nn.NLLLoss()

optimizer = torch.optim.SGD(model.parameters(),lr =   LR)

torch.manual_seed(SEED)
for epoch in range(EPOCHS):
    accuracy_hist_train = 0
    auroc_hist_train = 0.0
    loss_hist_train = 0
    for x_batch, y_batch in train_dl:
        pred = model(x_batch)
        loss = loss_fn(pred,y_batch)

        optimizer.zero_grad()
        loss.backward()

        loss_hist_train += loss.item() * len(y_batch)

        optimizer.step()

        optimizer.zero_grad()


        is_correct_1 = (pred.argmax(dim = 1) == y_batch)


        accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=10)
        is_correct_2 = accuracy(pred.argmax(dim = 1), y_batch).item() * len(y_batch)
        assert(is_correct_1.sum() ==  is_correct_2)

        accuracy_hist_train += is_correct_1.sum().item()

        auroc =torchmetrics.AUROC(task="multiclass", num_classes=10)
        auroc_hist_train += auroc(pred, y_batch) * BATCH_SIZE
    accuracy_hist_train /= len(train_dl.dataset)
    auroc_hist_train /= len(train_dl.dataset)
    loss_hist_train /= len(train_dl.dataset)
    print(f'Train Metrics Epoch {epoch} Loss {loss_hist_train:.4f} Accuracy {accuracy_hist_train:.4f} AUROC {auroc_hist_train:.4f}')

    accuracy_hist_test = 0
    auroc_hist_test = 0.0
    loss_hist_test = 00

    with torch.no_grad():
      accuracy_hist_test = 0
      auroc_hist_test = 0.0
      for x_batch, y_batch in test_dl:
          pred = model(x_batch)

          loss = loss_fn(pred,y_batch)

          loss_hist_test += x_batch.size(0) * loss.item()


          accuracy = Accuracy(task="multiclass", num_classes=10)
          is_correct =  accuracy(pred.argmax(dim = 1), y_batch).item() * len(y_batch)

          accuracy_hist_test +=  accuracy(pred.argmax(dim = 1), y_batch).item() * len(y_batch)

          auroc =torchmetrics.AUROC(task="multiclass", num_classes=10)
          auroc_hist_test +=  auroc(pred, y_batch) * BATCH_SIZE
      accuracy_hist_test /= len(test_dl.dataset)
      auroc_hist_test /= len(test_dl.dataset)
      loss_hist_test /= len(test_dl.dataset)
      print(f'Test Metrics Epoch {epoch} Loss {loss_hist_test:.4f} Accuracy {accuracy_hist_test:.4f} AUROC {auroc_hist_test:.4f}')

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028


Train Metrics Epoch 0 Loss 2.2644 Accuracy 0.2251 AUROC 0.7630
Test Metrics Epoch 0 Loss 2.2155 Accuracy 0.3305 AUROC 0.8787
Train Metrics Epoch 1 Loss 2.1683 Accuracy 0.4055 AUROC 0.8960
Test Metrics Epoch 1 Loss 2.1078 Accuracy 0.5435 AUROC 0.9134
Train Metrics Epoch 2 Loss 2.0493 Accuracy 0.5686 AUROC 0.9187
Test Metrics Epoch 2 Loss 1.9734 Accuracy 0.6048 AUROC 0.9312
Train Metrics Epoch 3 Loss 1.9050 Accuracy 0.6139 AUROC 0.9334
Test Metrics Epoch 3 Loss 1.8189 Accuracy 0.6390 AUROC 0.9444
Train Metrics Epoch 4 Loss 1.7482 Accuracy 0.6353 AUROC 0.9408
Test Metrics Epoch 4 Loss 1.6618 Accuracy 0.6526 AUROC 0.9492
Train Metrics Epoch 5 Loss 1.5959 Accuracy 0.6420 AUROC 0.9476
Test Metrics Epoch 5 Loss 1.5162 Accuracy 0.6528 AUROC 0.9556
Train Metrics Epoch 6 Loss 1.4587 Accuracy 0.6486 AUROC 0.9536
Test Metrics Epoch 6 Loss 1.3877 Accuracy 0.6642 AUROC 0.9588
Train Metrics Epoch 7 Loss 1.3386 Accuracy 0.6676 AUROC 0.9585
Test Metrics Epoch 7 Loss 1.2751 Accuracy 0.6939 AUROC 0.9626


In [ ]:

pred = model(mnist_train_dataset.data/255.)
is_correct =  (pred.argmax(dim = 1) == mnist_train_dataset.targets).float()
print(f'Total Final Train accuracy: {is_correct.mean():.4f}')

pred =  model(mnist_test_dataset.data/255.)
is_correct =  (pred.argmax(dim = 1) == mnist_test_dataset.targets).float()
print(f'Total Final Test accuracy: {is_correct.mean():.4f}')


Total Final Train accuracy: 0.8555
Total Final Test accuracy: 0.8593
